In [1]:
!git clone https://github.com/vncorenlp/VnCoreNLP
!pip install vncorenlp

fatal: destination path 'VnCoreNLP' already exists and is not an empty directory.


In [2]:
from vncorenlp import VnCoreNLP
import pandas as pd
import re

In [3]:
rdrsegmenter = VnCoreNLP("/content/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

In [7]:
macp = pd.read_excel('/content/Macp.xlsx')
macp = macp.dropna()
tenct = macp['Tên Công ty'].tolist()
for i in range(len(tenct)):
  tenct[i] = str(tenct[i]).lower()
tenct[:5]
tenma = macp['Mã '].tolist()

def del_test(text):
  year = ['năm 2021', 'năm 2020', 'năm 2019', 'năm 2018', 'năm 2017', 'năm 2016', 'năm 2015', 'năm 2014', 'năm 2013', 'năm 2012', 'năm 2011', 'năm 2010', 'Năm 2021', 'Năm 2020', 'Năm 2019', 'Năm 2018', 'Năm 2017', 'Năm 2016', 'Năm 2015', 'Năm 2014', 'Năm 2013', 'Năm 2012', 'Năm 2011', 'Năm 2010', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']
  month = ['tháng 1', 'tháng 2', 'tháng 3', 'tháng 4', 'tháng 5', 'tháng 6', 'tháng 7', 'tháng 8', 'tháng 9', 'tháng 10', 'tháng 11', 'tháng 12', 'Tháng 1', 'Tháng 2', 'Tháng 3', 'Tháng 4', 'Tháng 5', 'Tháng 6', 'Tháng 7', 'Tháng 8', 'Tháng 9', 'Tháng 10', 'Tháng 11', 'tháng 12']
  quy = ['quý 1', 'quý 2', 'quý 3', 'quý 4', 'Quý 1', 'Quý 2', 'Quý 3', 'Quý 4']
  text = text.replace('Covid-19', 'Covid')
  word_segmented_text = rdrsegmenter.ner(text)[0]
  for char, typ in word_segmented_text:
    if typ == 'B-ORG' or typ == 'I-ORG' or typ == 'B-PER' or typ == 'I-PER':
      char = char.replace('_', ' ')
      text = text.replace(char, 'name')
    if typ == "B-LOC" or typ == "I-LOC":
      if char != 'VN':
        char = char.replace('_', ' ')
        text = text.replace(char,'loc')
    if typ == 'O':
      if len(re.findall('\d*\.?\,?\d+\%', char)) > 0:
        text = text.replace(char, 'percent')
      if len(re.findall('\s?\(?[A-Z]{3,4}\)?\s?', char)) > 0 and char != 'USD':
          text = text.replace(char, 'name')
      if char in tenma:
        text = text.replace(char, 'name')
      char = char.replace('_', ' ')
      char_lower = char.lower()
      if char_lower in tenct:
        text = text.replace(char, 'name')
  text = text.replace('"', '')
  text = text.replace('”', '')
  text = text.replace('“', '')
  text = text.replace('.', '')
  text = text.replace(',', '')
  text = text.replace('(', '')
  text = text.replace(')', '')
  text = text.replace(':', '')
  text = text.replace('[', '')
  text = text.replace(']', '')
  text = text.replace('-', ' ')
  text = re.sub('\d{0,2}-?\d{0,2}\/\d{1,4}', 'date', text)
  for i in quy:
    text = text.replace(i, 'date')
  for i in year:
    text = text.replace(i, 'date')
  for i in month:
    text = text.replace(i, 'date')
  text = re.sub('\d+ năm ', 'date ', text)
  text = re.sub('\d+ tháng ', 'date ', text)
  text = re.sub(' \-?\d+\w?', ' number', text)
  text = text.split()
  for i in range(len(text)):
    if text[i].isdigit():
      text[i] = 'number'
  text = ' '.join(text)
  text1 = text.split()
  for i in range(len(text1)+1):
    try:
      if text1[i][0].isupper() and text1[i+1][0].isupper():
        text = text.replace(text1[i], 'name')
        text = text.replace(text1[i+1], 'name')
    except:
      pass
  text = rdrsegmenter.tokenize(text)
  text = ' '.join([' '.join(x) for x in text])
  text = text.lower()
  return text

In [5]:
data = pd.read_excel('raw_data.xlsx')
data.head(10)

,title,label
0,"""Trùm"" logistics Sotrans lên kế hoạch tăng gấp...",3
1,"ĐHĐCĐ SSI: ""Khen ngợi CTCK thị phần số 1, nhưn...",2
2,Cổ phiếu nhóm Viettel giảm mạnh dù thị trường ...,2
3,"Cổ phiếu lên đỉnh, con gái chủ tịch Haxaco (HA...",2
4,"FPT: Cổ phiếu bứt phá 59%, lãi ròng 4 tháng đầ...",3
5,"Soi lợi nhuận quý 1 của các cổ phiếu ""hàng hiệu""",2
6,"SP mua lại 49% cổ phần Skylar từ BCG Energy, x...",3
7,DIC Corp (DIG) dự kiến phát hành gần 60 triệu ...,3
8,Sự vực dậy sau hàng thập kỷ của những thương h...,3
9,8 sai lầm kể cả nhà đầu tư chứng khoán lâu năm...,2


In [ ]:
new_title = []
title = data['title'].tolist()
for i in title:
  new_title.append(del_test(i))
new_title

In [11]:
new_data = pd.DataFrame()
new_data['title'] = new_title
new_data['label'] = data['label']
new_data.head()

,title,label
0,trùm logistics name lên kế_hoạch tăng gấp đôi ...,3
1,đhđcđ name khen_ngợi name thị_phần số number n...,2
2,cổ_phiếu nhóm name giảm mạnh dù thị_trường lên...,2
3,cổ_phiếu lên đỉnh con gái chủ_tịch name name đ...,2
4,name cổ_phiếu bứt_phá percent lãi_ròng date đầ...,3


In [18]:
count = 0
title = new_data['title'].to_list()
label = new_data['label'].to_list()
with open('test-3.txt', 'w') as obj:
  for i in range(new_data.shape[0]):
    obj.write('train_{}'.format(count))
    obj.write('\n')
    obj.write(title[i])
    obj.write('\n')
    obj.write(str(label[i]))
    obj.write('\n\n')
    count += 1